In [ ]:
#%%appyter init
import os, sys; sys.path.insert(0, os.path.realpath('..'))
from appyter import magic
magic.init(lambda _=globals: _())

In [ ]:
%matplotlib inline
# Imports
## Data processing
import pandas as pd
import numpy as np
import scipy as sp
## Machine Learning
import sklearn as sk
from sklearn import (
    calibration,
    decomposition,
    ensemble,
    feature_selection,
    linear_model,
    manifold,
    metrics,
    model_selection,
    multioutput,
    pipeline,
    preprocessing,
    svm,
    tree,
    feature_extraction,
    neural_network,
)
from split import StratifiedGroupKFold, RepeatedStratifiedGroupKFold
import umap
## Plotting
from matplotlib import pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
## Drugmonizome API
from drugmonizome import Drugmonizome
## SEP-L1000 data retrieval
from sepl1000 import SEPL1000
## L1000FWD queries
import querysepl1000fwd
## Match drug name inputs using PubChem API
from DrugNameConverter import DrugNameConverter
# Utility
import os
import re
import json
from functools import reduce
from IPython.display import display
from tqdm import tqdm
# Interactive tables
from itables import show

In [ ]:
rng = 2020
np.random.seed(rng)

## Select Input Datasets and Target Classes

Selected drug set libraries and omics datasets are downloaded and joined on the drug to produce a large association matrix. A machine learning model will be trained to predict the specified target labels from this association matrix. This is a binary classification task that can be used to predict drugs that are likely to be associated with the target class.

In [ ]:
%%appyter hide
{% do SectionField(
    title='Input Dataset Selection',
    subtitle='Select the input datasets to use for learning and classification. \
              A model will be trained to predict the target labels from the selected features. \
              If no datasets are selected, default features will be used.',
    name='ATTRIBUTES',
    img='attributes.png',
) %}

{% set sepl1000datasets = MultiCheckboxField(
    name='sepl1000datasets',
    label='Transcriptomic and Imaging Datasets after Perturbation<br>(from the SEP-L1000 project)',
    description='These input datasets were used previously for side effect prediction (https://maayanlab.net/SEP-L1000/).',
    choices=[
        'LINCS Gene Expression Signatures',
        'GO Transformed Signatures (PAEA)',
        'MLPCN Cell Morphological Profiling',
        'MACCS Chemical Fingerprint',
    ],
    default=[],
    section='ATTRIBUTES'
) %}

{% set exprdatasets = MultiCheckboxField(
    name='exprdatasets',
    label='L1000FWD<br>(drug set libraries from Drugmonizome)',
    description='Top up and down-regulated genes after perturbation, along with enriched pathways.',
    choices=[
        'L1000FWD Downregulated GO Biological Processes',
        'L1000FWD Downregulated GO Cellular Components',
        'L1000FWD Downregulated GO Molecular Function',
        'L1000FWD Downregulated KEGG Pathways',
        'L1000FWD Downregulated Signatures',
        'L1000FWD Predicted Side Effects',
        'L1000FWD Upregulated GO Biological Process',
        'L1000FWD Upregulated GO Cellular Components',
        'L1000FWD Upregulated GO Molecular Function',
        'L1000FWD Upregulated KEGG Pathways',
        'L1000FWD Upregulated Signatures',
    ],
    default=[],
    section='ATTRIBUTES'
) %}

{% set targetdatasets = MultiCheckboxField(
    name='targetdatasets',
    label='Drug Targets and Associated Genes<br>(drug set libraries from Drugmonizome)',
    choices=[
        'Downregulated CREEDS Signatures',
        'Upregulated CREEDS Signatures',
        'DrugCentral Targets',
        'DrugRepurposingHub Drug Targets',
        'Drugbank Small Molecule Carriers',
        'Drugbank Small Molecule Enzymes',
        'Drugbank Small Molecule Targets',
        'Drugbank Small Molecule Transporters',
        'Geneshot Associated Genes',
        'Geneshot Predicted AutoRIF Genes',
        'Geneshot Predicted Coexpression Genes',
        'Geneshot Predicted Enrichr Genes',
        'Geneshot Predicted GeneRIF Genes',
        'Geneshot Predicted Tagger Genes',
        'KinomeScan Kinases',
        'PharmGKB Single Nucleotide Polymorphisms',
        'STITCH Targets',
    ],
    default=[],
    section='ATTRIBUTES'
) %}

{% set indicationdatasets = MultiCheckboxField(
    name='indicationdatasets',
    label='Indications, Modes of Action, and Side Effects<br>(drug set libraries from Drugmonizome)',
    choices=[
        'ATC Codes Drugsetlibrary',
        'DrugRepurposingHub Mechanisms of Action',
        'PharmGKB OFFSIDES Side Effects',
        'SIDER Indications',
        'SIDER Side Effects',
    ],
    default=[],
    section='ATTRIBUTES'
) %}

{% set structuraldatasets = MultiCheckboxField(
    name='structuraldatasets',
    label='Structural Features<br>(drug set libraries from Drugmonizome)',
    choices=[
        'RDKIT MACCS Chemical Fingerprints'
    ],
    default=[],
    section='ATTRIBUTES'
) %}

{% set keepmissing = BoolField(
    name='keepmissing',
    label='Keep drugs with missing data when joining datasets',
    description='Keep drugs that appear in some datasets and not in others. \
                 Missing data is filled in with zeros. Otherwise, only drugs \
                 that are present in all datasets are preserved.',
    default=False,
    section='ATTRIBUTES',
) %}

{% set tfidf = BoolField(
    name='tfidf',
    label='Apply tf–idf normalization to binary inputs',
    description='For binary drug-attribute associations in the input matrix, \
                 apply tf-idf transformation to normalize data.',
    default=True,
    section='ATTRIBUTES',
) %}

{% set attribute_datasets = exprdatasets.value +
                             targetdatasets.value +
                             indicationdatasets.value +
                             structuraldatasets.value %}

In [ ]:
%%appyter markdown

To construct the input matrix, we download drug set libraries and omics datasets and join them on the InChI Key.
{% if keepmissing.value %} Drugs that appear in some datasets and not in others are retained, and missing data is filled in with zeros.
{% else %} Only drugs that are present in all datasets are retained.
{% endif %}

In [ ]:
%%appyter hide
{% do SectionField(
    title='Target Label Selection',
    subtitle='Upload a list of compounds or select an attribute from Drugmonizome to be assigned a positive class label for binary classification.',
    name='TARGET',
    img='target.png',
) %}

{% set target_field = TargetField(
    name='target_field',
    label='Target Selection',
    default='Attribute',
    description='Select input method',
    choices={
        'List': [
            ChoiceField(
                name='drugformat',
                label='Drug Identifier Format',
                description='Compounds can be specified by either drug name or InChI Key.',
                default='InChI Key',
                choices=[
                    'Drug Name',
                    'InChI Key'
                ],
                section='TARGET'
            ),
            FileField(
                name='drughitlist',
                label='Upload List of Compounds',
                description='Upload a list of compounds to be assigned positive class labels for binary classification. \
                             Compounds should be in a text file, specified by either drug name or InChI Key and separated by newlines.',
                default='COVID19ScreenHitsInChIKeys.txt',
                examples={
                    'COVID19ScreenHits.txt': 'https://appyters.maayanlab.cloud/storage/Drugmonizome_ML/COVID19ScreenHits.txt',
                    'COVID19ScreenHitsInChIKeys.txt': 'https://appyters.maayanlab.cloud/storage/Drugmonizome_ML/COVID19ScreenHitsInChIKeys.txt',
                },
                section='TARGET'
            ),
        ],
        'Attribute': [
            AutocompleteField(
                name='target_attribute',
                description='Enter a small molecule attribute from one of the Drugmonizome datasets that should be predicted.',
                file_path="https://appyters.maayanlab.cloud/storage/Drugmonizome_ML/drugmonizome_terms.json",
                label='Attribute',
                hint='Enter Drugmonizome term...',
                default='',
                constraint='(^(.+) \\(from (.+)\\)$|^$)',
        )],
    },
    section='TARGET',
) %}

{% set includestereo = BoolField(
    name='includestereo',
    label='Include stereoisomers',
    description='If true, compounds are matched to entries in the datasets by the first 14 characters of their InChI Keys, \
                 so stereoisomers of the compounds in the input list or with a particular attritube are also counted as hits. \
                 Note that different resources record different details for charge and stereochemistry, \
                 causing some compounds to have different full-length InChI Keys in different datasets. \
                 Selecting this option may allow such drugs to be better matched to entries in the datasets.',
    default=True,
    section='TARGET',
) %}

{% set target_name, target_dataset = '', '' %}
{% if TargetField.raw_value == 'Attribute' %}
{% set target_name, target_dataset = target_field.value[0].value|re_match('^(.+) \\(from (.+)\\)$') %}
{% endif %}

In [ ]:
%%appyter code_exec

{% if sepl1000datasets.value == [] and attribute_datasets == [] %}
# No datasets selected, so use default dataset
sepl1000datasets = ['LINCS Gene Expression Signatures']
{% else %}
# Use the selected SEP-L1000 datasets
sepl1000datasets = {{ sepl1000datasets }}
{% endif %}

{% if sepl1000datasets.value != [] or attribute_datasets == [] %}
name_to_file = {
    'LINCS Gene Expression Signatures': 'LINCS_Gene_Experssion_signatures_CD.csv.gz',
    'GO Transformed Signatures (PAEA)': 'GO_transformed_signatures_PAEA.csv.gz',
    'MLPCN Cell Morphological Profiling': 'MLPCN_morplological_profiles.csv.gz',
    'MACCS Chemical Fingerprint': 'MACCS_bitmatrix.csv.gz',
}

df_sepl1000_list = list(SEPL1000.download_df(list(name_to_file[dataset] for dataset in sepl1000datasets),
                                             index_col=0))
dataset_sizes = list(zip(sepl1000datasets, [dataset.shape[1] for dataset in df_sepl1000_list]))

# Assemble all SEP-L1000 datasets
if len(df_sepl1000_list) > 1:
    # Obtain merged dataframe with omics and target data
    df_sepl1000 = reduce(
        lambda a, b: pd.merge( # Merge two dataframes item by item
            a, # left
            b, # right
            # Items with the same left and right index are merged
            left_index=True,
            right_index=True,
            {% if keepmissing.value %}
            how='outer', # Keep mis-matched indices
            {% else %}
            how='inner', # Keep only matched indices
            {% endif %}
        ),
        df_sepl1000_list,
    )
else:
    df_sepl1000 = df_sepl1000_list[0]

del(df_sepl1000_list)

# Mean-fill infinite and missing values
df_sepl1000 = df_sepl1000.replace([np.inf, -np.inf], np.nan)
df_sepl1000 = df_sepl1000.fillna(np.mean(df_sepl1000))
print('Total shape:', df_sepl1000.shape)
display(df_sepl1000.head())
{% else %}
dataset_sizes = []
{% endif %}

In [ ]:
%%appyter code_exec

# Use the selected attribute datasets
attribute_datasets = {{ attribute_datasets }}

{% if attribute_datasets == [] %}
X = df_sepl1000
{% else %}
df_attributes = list(Drugmonizome.download_df(
    [dataset
     for dataset in attribute_datasets]
))
dataset_sizes += list(zip(attribute_datasets, [dataset.shape[1] for dataset in df_attributes]))

# Assemble all attribute datasets
if len(df_attributes) > 1:
    # Obtain merged dataframe with omics and target data
    df = reduce(
        lambda a, b: pd.merge( # Merge two dataframes item by item
            a, # left
            b, # right
            # Items with the same left and right index are merged
            left_index=True,
            right_index=True,
            {% if keepmissing.value %}
            how='outer', # Keep mis-matched indices
            {% else %}
            how='inner', # Keep only matched indices
            {% endif %}
        ),
        df_attributes,
    )
else:
    df = df_attributes[0]

del(df_attributes)

df = df.fillna(0)
X = df.applymap(lambda f: 1 if f!=0 else 0)
{% if tfidf.value %}
# Apply tf-idf normalization
transformer = feature_extraction.text.TfidfTransformer()
X_tfidf = transformer.fit_transform(X).toarray()
X = pd.DataFrame(X_tfidf, columns=X.columns, index=X.index)
{% if sepl1000datasets.value != [] %}
{% if keepmissing.value %}
X = pd.merge(df_sepl1000, X, left_index=True, right_index=True, how='outer') # Keep mis-matched indices
{% else %}
X = pd.merge(df_sepl1000, X, left_index=True, right_index=True) # Keep only matched indices
{% endif %}
{% endif %}
{% endif %}
{% endif %}

print('Total shape:', X.shape)
display(X.head())

In [ ]:
%%appyter markdown

{% if target_field.raw_value == 'List' %}
The target labels are produced from the uploaded list of hits: 1 if the drug is specified as a hit, 0 otherwise.
{% if target_field.value[0].value == 'Drug Name' %} Drug names are matched to InChI Keys from the Drugmonizome metadata.
{% endif %}
{% endif %}

In [ ]:
%%appyter code_exec

{% if target_field.raw_value == 'List' %}
{% if target_field.value[1].value == '' %}
# Using default list of hits from COVID-19 in vitro drug screens
hits_filename = '../../COVID19ScreenHits.txt'
{% else %}
# Using user-specified list of positive drug hits
hits_filename = {{target_field.value[1]}}
{% endif %}

{% if target_field.value[0].value == 'InChI Key' %}
# Read InChI Keys from file
with open(hits_filename, 'r') as hits_file:
    drug_hits = set(drug.strip().upper() for drug in hits_file.read().strip().split('\n') 
                    if len(drug.strip()) > 0)

{% elif target_field.value[0].value == 'Drug Name' %}
# Helper functions
def merge(A, B, f):
    """
    Merges two dictionaries, where items from shared keys are merged using a custom function.
    """
    merged = {k: A.get(k, B.get(k)) for k in A.keys() ^ B.keys()}
    merged.update({k: f(A[k], B[k]) for k in A.keys() & B.keys()})
    return merged

def save_items(out_file, items):
    """
    Saves list of items as rows in a file.
    """
    with open(out_file, 'w') as f:
        for i in range(len(items)):
            if i < len(items) - 1:
                f.write(items[i] + '\n')
            else:
                f.write(items[i])

def save_gmt(out_file, keys_to_sets, sep='\t'):
    """
    Saves dict with key-set pairs as gmt file format.
    """
    lines = []
    for key in sorted(keys_to_sets):
        lines.append(key + sep*2 + sep.join(sorted(keys_to_sets[key])))
    save_items(out_file, lines)

# Read drug names from file
with open(hits_filename, 'r') as hits_file:
    drug_hits = set(drug.strip().lower() for drug in hits_file.read().strip().split('\n') 
                    if len(drug.strip()) > 0)

# Query PubChem API to map drug names to InChI Keys
print('Querying PubChem API...')
drug_hits_inchi_pubchem = DrugNameConverter.batch_to_inchi_keys(drug_hits)
# Query Drugmonizome API to map drug names to InChI Keys
print('Querying Drugmonizome API...')
drug_hits_inchi_drugmonizome = Drugmonizome.map_names_to_inchi_keys(drug_hits)
# Query L1000FWD API to map drug names to InChI Keys
print('Querying L1000FWD API...')
drug_hits_inchi_l1000fwd = querysepl1000fwd.map_names_to_inchi_keys(drug_hits)

# Combine InChI Keys from all resources
drug_hits_inchi = merge(drug_hits_inchi_pubchem, drug_hits_inchi_drugmonizome, lambda s1, s2: s1 | s2)
drug_hits_inchi = merge(drug_hits_inchi, drug_hits_inchi_l1000fwd, lambda s1, s2: s1 | s2)
save_gmt('hits_drug_name_to_inchi_keys.gmt', drug_hits_inchi)
# Unmatched drug names
unmatched_drugs = set(drug for drug in drug_hits
                      if drug not in drug_hits_inchi or len(drug_hits_inchi[drug]) == 0)
print(f'Drugs without InChI Keys ({ len(unmatched_drugs) }/{ len(drug_hits) }):', unmatched_drugs)

# Set of InChI Keys for user-specified hits
drug_hits = set(key for drug in drug_hits_inchi
                    for key in drug_hits_inchi[drug])
save_items('hits_inchi_keys.txt', sorted(drug_hits))
{% endif %}

{% else %}

df_target = list(Drugmonizome.download_df(
    ['{{ target_dataset }}']
))
df = df_target[0]
df = df.fillna(0)
Y = df.applymap(lambda f: 1 if f!=0 else 0)
drug_hits = set(Y[Y['{{ target_name }}'] == 1].index)

# Helper function
def save_items(out_file, items):
    """
    Saves list of items as rows in a file.
    """
    with open(out_file, 'w') as f:
        for i in range(len(items)):
            if i < len(items) - 1:
                f.write(items[i] + '\n')
            else:
                f.write(items[i])
save_items('hits_inchi_keys.txt', sorted(drug_hits))
{% endif %}

In [ ]:
%%appyter markdown

{% if target_field.raw_value == 'List' %}
{% if target_field.value[0].value == 'Drug Name' %}
For the user-inputted drug names:
* Mapping of drug name to InChI Key: [hits_drug_name_to_inchi_keys.gmt](./hits_drug_name_to_inchi_keys.gmt)
* List of InChI Keys: [hits_inchi_keys.txt](./hits_inchi_keys.txt)
{% endif %}
{% endif %}

We produce a target array containing 1 if the drug is specified as a hit and 0 otherwise.

In [ ]:
%%appyter code_exec

{% if includestereo.value %}
# Match first 14 characters of InChI Keys (hash of InChI connectivity information)
drug_hits_inchi_main_layer = set(key[:14] for key in drug_hits)
y = np.array([drug[:14] in drug_hits_inchi_main_layer for drug in X.index]).astype(np.int8)
{% else %}
# Match full InChI Keys
y = np.array([drug in drug_hits for drug in X.index]).astype(np.int8)
{% endif %}
print('Number of hits matched in input: %d (%0.3f %%)' % (y.sum(), 100*y.sum()/len(y)))

In [ ]:
# Output data shapes
print('Input shape:', X.shape)
print('Target shape:', y.shape)

## Dimensionality Reduction and Visualization

In [ ]:
%%appyter hide
{% do SectionField(
    title='Machine Learning Pipeline',
    subtitle='Select from available machine learning algorithms, their unique settings, and methods to use to evaluate the classifier.',
    name='SETTINGS',
    img='settings.png',
) %}

{% set visualization_reduction = ChoiceField(
    name='visualization_reduction',
    label='Data Visualization Method',
    description='Select a dimensionality reduction algorithm for data visualization.',
    default='UMAP',
    choices={
        'UMAP': 'umap.UMAP(random_state=rng)',
        'NMF': 'sk.decomposition.NMF(n_components=2)',
        'PCA': 'sk.decomposition.PCA(n_components=2)',
        'TruncatedSVD': 'sk.decomposition.TruncatedSVD(n_components=2)',
        'IncrementalPCA': 'sk.decomposition.IncrementalPCA(n_components=2)',
        'ICA': 'sk.decomposition.FastICA(n_components=2)',
        'SparsePCA': 'sk.decomposition.SparsePCA(n_components=2)',
    },
    section='SETTINGS'
) %}

In [ ]:
%%appyter markdown

We reduce the dimensionality of our omics feature space for visualization with {{ visualization_reduction.raw_value }}.

In [ ]:
%%appyter code_exec
clf_dimensionality_reduction = {{ visualization_reduction }}
X_reduced = clf_dimensionality_reduction.fit_transform(X.values)
{% if visualization_reduction.raw_value == 'PCA' %}
print('Explained variance:', np.sum(clf_dimensionality_reduction.explained_variance_))
{% endif %}

In [ ]:
X_reduced_df = pd.DataFrame(X_reduced, columns=['Component 1', 'Component 2'])
X_reduced_df['Drug Name'] = querysepl1000fwd.get_drug_names(X.index)
X_reduced_df['InChI Key'] = X.index
X_reduced_df['Label'] = y
X_reduced_df['marker symbol'] = ['x' if label else 'circle' for label in X_reduced_df['Label']]
X_reduced_df['text'] = ['<br>'.join(['Drug Name: ' + str(name),
                                     'InChI Key: ' + str(inchi),
                                     'Label: ' + str(label)])
                        for name, inchi, label in zip(X_reduced_df['Drug Name'],
                                                      X_reduced_df['InChI Key'],
                                                      X_reduced_df['Label'])]

In [ ]:
%%appyter code_exec

fig = go.Figure()
for label in set(X_reduced_df['Label']):
    X_plot = X_reduced_df[X_reduced_df['Label'] == label].sort_values('Label')
    fig.add_trace(go.Scatter(mode='markers',
                             x=X_plot['Component 1'], y=X_plot['Component 2'],
                             text=X_plot['text'],
                             name=label,
                             marker=dict(
                                 color=['#0d0887', '#f0f921'][label%2],
                                 size=8,
                                 symbol=X_plot['marker symbol'],
                                 line_width=1,
                                 line_color='white'
                             )))
fig.update_layout(height=600, width=800,
                  xaxis_title='Component 1',
                  yaxis_title='Component 2',
                  title_text='Known Labels ({{ visualization_reduction.raw_value }})',
                  legend_title_text='Target Label',
                  template='simple_white')
fig.show()

## Machine Learning

In [ ]:
%%appyter hide
{% set dimensionality_reduction = ChoiceField(
    name='dimensionality_reduction',
    label='Dimensionality Reduction Algorithm',
    description='Optionally select a dimensionality reduction algorithm as a data preprocessing step in the ML pipeline.',
    default='None',
    choices={
        'None': 'None',
        'PCA': 'sk.decomposition.PCA(n_components=64)',
        'TruncatedSVD': 'sk.decomposition.TruncatedSVD(n_components=64)',
        'IncrementalPCA': 'sk.decomposition.IncrementalPCA(n_components=64)',
        'ICA': 'sk.decomposition.FastICA(n_components=64)',
        'SparsePCA': 'sk.decomposition.SparsePCA(n_components=64)',
    },
    section='SETTINGS'
) %}
{% set feature_selection = ChoiceField(
    name='feature_selection',
    label='Machine Learning Feature Selection',
    description='Optionally select a feature selection algorithm to include in the ML pipeline. \
                 If RecursiveSelectionFromExtraTrees is chosen, additional information can be obtained \
                 on the relative importance of different features based on which features are eliminated.',
    default='None',
    choices={
        'None': 'None',
        'SelectFromLinearSVC': 'sk.feature_selection.SelectFromModel(sk.svm.LinearSVC(loss="squared_hinge", penalty="l1", dual=False, class_weight="balanced"))',
        'SelectFromExtraTrees': 'sk.feature_selection.SelectFromModel(sk.ensemble.ExtraTreesClassifier(class_weight="balanced"))',
        'RecursiveSelectionFromExtraTrees': 'sk.feature_selection.RFE(sk.ensemble.ExtraTreesClassifier(class_weight="balanced"), n_features_to_select=256, step=0.1)',
        'SelectKBest': 'sk.feature_selection.SelectKBest("f_classif")',
        'SelectKBestChi2': 'sk.feature_selection.SelectKBest("chi2")',
        'SelectKBestMultiInfo': 'sk.feature_selection.SelectKBest("mutual_info_classif")',
    },
    section='SETTINGS'
) %}
{% set algorithm = TargetField(
    name='algorithm',
    label='Machine Learning Algorithm',
    default='ExtraTreesClassifier',
    description='Select a machine learning algorithm to construct the predictive model. \
                 (See scikit-learn User Guide for details.)',
    choices={
        'GradientBoostingClassifier': [
            ChoiceField(
                name='loss_gb',
                label='loss',
                description='Loss function to be optimized.',
                default="deviance",
                choices=["deviance", "exponential"],
            ),
            FloatField(
                name='learning_rate_gb',
                label='learning_rate',
                description='Shrinks the contribution of each tree by learning_rate.',
                default=0.1,
            ),
            IntField(
                name='n_estimators_gb',
                label='n_estimators',
                description='Number of boosting stages to perform.',
                default=100,
            ),
            FloatField(
                name='subsample_gb',
                label='subsample',
                description='Fraction of samples to be used for fitting individual base learners.',
                default=1.0,
            ),
            ChoiceField(
                name='criterion_gb',
                label='criterion',
                description='Function to measure the quality of a split.',
                default="friedman_mse",
                choices=["friedman_mse", "mse", "mae"],
            ),
            FloatField(
                name='tol_gb',
                label='tol',
                description='Tolerance for early stopping.',
                default=1e-4,
            ),
        ],
        'RandomForestClassifier': [
            IntField(
                name='n_estimators_rf',
                label='n_estimators',
                description='Number of trees in the forest.',
                default=100,
            ),
            ChoiceField(
                name='criterion_rf',
                label='criterion',
                description='Function to measure the quality of a split.',
                default="gini",
                choices=["gini", "entropy"],
            ),
            FloatField(
                name='min_samples_split_rf',
                label='min_samples_split',
                description='Minimum number of samples required to split an internal node. \
                             If int, then min_samples_split specifies the minimum number. \
                             If float, then min_samples_split specifies a fraction of the total number of samples.',
                default=2,
            ),
            FloatField(
                name='min_samples_leaf_rf',
                label='min_samples_leaf',
                description='Minimum number of samples required to be at a leaf node. \
                             If int, then min_samples_leaf specifies the minimum number. \
                             If float, then min_samples_leaf specifies a fraction of the total number of samples.',
                default=1,
            ),
            ChoiceField(
                name='max_features_rf',
                label='max_features',
                description='The number of features to consider when looking for the best split.',
                default="None",
                choices=["None", '"auto"', '"sqrt"', '"log2"'],
            ),
            FloatField(
                name='min_impurity_decrease_rf',
                label='min_impurity_decrease',
                description='A node will be split if this split induces a decrease of the impurity greater than or equal to this value.',
                default=0.0,
            ),
            ChoiceField(
                name='class_weight_rf',
                label='class_weight',
                description='Weights associated with classes. If None, then all classes have weight one. \
                             The balanced mode adjusts weights inversely proportional to class frequencies in the input data. \
                             The balanced_subsample mode is the same as balanced except weights are computed based on the bootstrap sample for each tree.',
                default='"balanced"',
                choices=["None", '"balanced"', '"balanced_subsample"'],
            ),
            FloatField(
                name='ccp_alpha_rf',
                label='ccp_alpha',
                description='Complexity parameter used for Minimal Cost-Complexity Pruning. \
                             The subtree with the largest cost complexity that is smaller than ccp_alpha will be chosen. \
                             By default, no pruning is performed.',
                default=0.0,
            ),
        ],
        'AdaBoostClassifier': [
            IntField(
                name='max_depth_ab',
                label='max_depth',
                description='Maximum depth of the decision tree used as the base estimator.',
                default=1,
            ),
            IntField(
                name='n_estimators_ab',
                label='n_estimators',
                description='Maximum number of estimators at which boosting is terminated.',
                default=50,
            ),
            FloatField(
                name='learning_rate_ab',
                label='learning_rate',
                description='Shrinks the contribution of each classifier by learning_rate.',
                default=1.0,
            ),
            ChoiceField(
                name='algorithm_ab',
                label='algorithm',
                description='Select the real or discrete boosting algorithm to use.',
                default="SAMME.R",
                choices=["SAMME", "SAMME.R"],
            ),
        ],
        'ExtraTreesClassifier': [
            IntField(
                name='n_estimators_et',
                label='n_estimators',
                description='Number of trees in the forest.',
                default=100,
            ),
            ChoiceField(
                name='criterion_et',
                label='criterion',
                description='Function to measure the quality of a split.',
                default="gini",
                choices=["gini", "entropy"],
            ),
            FloatField(
                name='min_samples_split_et',
                label='min_samples_split',
                description='Minimum number of samples required to split an internal node. \
                             If int, then min_samples_split specifies the minimum number. \
                             If float, then min_samples_split specifies a fraction of the total number of samples.',
                default=2,
            ),
            FloatField(
                name='min_samples_leaf_et',
                label='min_samples_leaf',
                description='Minimum number of samples required to be at a leaf node. \
                             If int, then min_samples_leaf specifies the minimum number. \
                             If float, then min_samples_leaf specifies a fraction of the total number of samples.',
                default=1,
            ),
            ChoiceField(
                name='max_features_et',
                label='max_features',
                description='The number of features to consider when looking for the best split.',
                default="None",
                choices=["None", '"auto"', '"sqrt"', '"log2"'],
            ),
            FloatField(
                name='min_impurity_decrease_et',
                label='min_impurity_decrease',
                description='A node will be split if this split induces a decrease of the impurity greater than or equal to this value.',
                default=0.0,
            ),
            ChoiceField(
                name='class_weight_et',
                label='class_weight',
                description='Weights associated with classes. If None, then all classes have weight one. \
                             The balanced mode adjusts weights inversely proportional to class frequencies in the input data. \
                             The balanced_subsample mode is the same as balanced except weights are computed based on the bootstrap sample for each tree.',
                default='"balanced"',
                choices=["None", '"balanced"', '"balanced_subsample"'],
            ),
            FloatField(
                name='ccp_alpha_et',
                label='ccp_alpha',
                description='Complexity parameter used for Minimal Cost-Complexity Pruning. \
                             The subtree with the largest cost complexity that is smaller than ccp_alpha will be chosen. \
                             By default, no pruning is performed.',
                default=0.0,
            ),
        ],
        'DecisionTreeClassifier': [
            ChoiceField(
                name='criterion_dt',
                label='criterion',
                description='Function to measure the quality of a split.',
                default="gini",
                choices=["gini", "entropy"],
            ),
            ChoiceField(
                name='splitter_dt',
                label='splitter',
                description='Strategy used to choose the split at each node.',
                default="best",
                choices=["best", "random"],
            ),
            FloatField(
                name='min_samples_split_dt',
                label='min_samples_split',
                description='Minimum number of samples required to split an internal node. \
                             If int, then min_samples_split specifies the minimum number. \
                             If float, then min_samples_split specifies a fraction of the total number of samples.',
                default=2,
            ),
            FloatField(
                name='min_samples_leaf_dt',
                label='min_samples_leaf',
                description='Minimum number of samples required to be at a leaf node. \
                             If int, then min_samples_leaf specifies the minimum number. \
                             If float, then min_samples_leaf specifies a fraction of the total number of samples.',
                default=1,
            ),
            ChoiceField(
                name='max_features_dt',
                label='max_features',
                description='The number of features to consider when looking for the best split.',
                default="None",
                choices=["None", '"auto"', '"sqrt"', '"log2"'],
            ),
            FloatField(
                name='min_impurity_decrease_dt',
                label='min_impurity_decrease',
                description='A node will be split if this split induces a decrease of the impurity greater than or equal to this value.',
                default=0.0,
            ),
            ChoiceField(
                name='class_weight_dt',
                label='class_weight',
                description='Weights associated with classes. If None, then all classes have weight one. \
                             The balanced mode adjusts weights inversely proportional to class frequencies in the input data. \
                             The balanced_subsample mode is the same as balanced except weights are computed based on the bootstrap sample for each tree.',
                default='"balanced"',
                choices=["None", '"balanced"', '"balanced_subsample"'],
            ),
            FloatField(
                name='ccp_alpha_dt',
                label='ccp_alpha',
                description='Complexity parameter used for Minimal Cost-Complexity Pruning. \
                             The subtree with the largest cost complexity that is smaller than ccp_alpha will be chosen. \
                             By default, no pruning is performed.',
                default=0.0,
            ),
        ],
        'KNeighborsClassifier': [
            IntField(
                name='n_neighbors_knn',
                label='n_neighbors',
                description='Number of neighbors to use for queries.',
                default=5,
            ),
            ChoiceField(
                name='weights_knn',
                label='weights',
                description='Weight function used in prediction. \
                             If uniform, all points in each neighborhood are weighted equally. \
                             If distance, points are weighted by the inverse of their distance.',
                default="uniform",
                choices=["uniform", "distance"],
            ),
            ChoiceField(
                name='algorithm_knn',
                label='algorithm',
                description='Algorithm used to compute the nearest neighbors.',
                default="auto",
                choices=["auto", "ball_tree", "kd_tree", "brute"],
            ),
            IntField(
                name='leaf_size_knn',
                label='leaf_size',
                description='Leaf size passed to BallTree or KDTree.',
                default=30,
            ),
            IntField(
                name='p_knn',
                label='p',
                description='Power parameter for the Minkowski metric.',
                default=2,
            ),
            ChoiceField(
                name='metric_knn',
                label='metric',
                description='Distance metric to use for the tree.',
                default="minkowski",
                choices=["minkowski", "euclidean", "manhattan", "chebyshev"],
            ),
        ],
        'RadiusNeighborsClassifier': [
            FloatField(
                name='radius_rn',
                label='radius',
                description='Range of parameter space to use for queries.',
                default=1.0,
            ),
            ChoiceField(
                name='weights_rn',
                label='weights',
                description='Weight function used in prediction. \
                             If uniform, all points in each neighborhood are weighted equally. \
                             If distance, points are weighted by the inverse of their distance.',
                default="uniform",
                choices=["uniform", "distance"],
            ),
            ChoiceField(
                name='algorithm_rn',
                label='algorithm',
                description='Algorithm used to compute the nearest neighbors.',
                default="auto",
                choices=["auto", "ball_tree", "kd_tree", "brute"],
            ),
            IntField(
                name='leaf_size_rn',
                label='leaf_size',
                description='Leaf size passed to BallTree or KDTree.',
                default=30,
            ),
            IntField(
                name='p_rn',
                label='p',
                description='Power parameter for the Minkowski metric.',
                default=2,
            ),
            ChoiceField(
                name='metric_rn',
                label='metric',
                description='Distance metric to use for the tree.',
                default="minkowski",
                choices=["minkowski", "euclidean", "manhattan", "chebyshev"],
            ),
        ],
        'MLPClassifier': [
            StringField(
                name='hidden_layer_sizes_mlp',
                label='hidden_layer_sizes',
                description='Enter a tuple, where the ith element represents the number of neurons in the ith hidden layer.',
                hint='Enter a tuple: e.g. (128, 64)',
                default='(100,)',
                constraint='^\\(\\s*(?:\\d+,\\s*)+(?:\\d+,?\\s*)?\\)$',
            ),
            ChoiceField(
                name='activation_mlp',
                label='activation',
                description='Activation function for the hidden layer.',
                default="relu",
                choices=["identity", "logistic", "tanh", "relu"],
            ),
            ChoiceField(
                name='solver_mlp',
                label='solver',
                description='Solver for weight optimization.',
                default="adam",
                choices=["lbfgs", "sgd", "adam"],
            ),
            FloatField(
                name='alpha_mlp',
                label='alpha',
                description='L2 penality (regularization term) parameter.',
                default=0.0001,
            ),
            ChoiceField(
                name='learning_rate_mlp',
                label='learning_rate',
                description='Learning rate schedule for weight updates. Only used for sgd solver.',
                default="constant",
                choices=["constant", "invscaling", "adaptive"],
            ),
            FloatField(
                name='learning_rate_init_mlp',
                label='learning_rate_init',
                description='The initial learning rate used. Controls the step-size in updating the weights. Only used for sgd or adam solver.',
                default=0.001,
            ),
            FloatField(
                name='power_t_mlp',
                label='power_t',
                description='Exponent for inverse scaling learning rate. Only used for sgd solver with invscaling for learning_rate.',
                default=0.5,
            ),
            IntField(
                name='max_iter_mlp',
                label='max_iter',
                description='Maximum number of iterations. The solver iterates until convergence (determined by tol) or this number of iterations.',
                default=200,
            ),
            FloatField(
                name='tol_mlp',
                label='tol',
                description='Tolerance for the optimization.',
                default=1e-4,
            ),
            BoolField(
                name='early_stopping_mlp',
                label='early_stopping',
                description='Whether to use early stopping to terminate training when validation score is not improving.',
                default=False,
            ),
            FloatField(
                name='validation_fraction_mlp',
                label='validation_fraction',
                description='The proportion of training data to set aside as validation set for early stopping.',
                default=0.1,
            ),
        ],
        'OneClassSVM': [
            ChoiceField(
                name='kernel_svm',
                label='kernel',
                description='Specifies the kernel type to be used in the algorithm.',
                default="rbf",
                choices=["linear", "poly", "rbf", "sigmoid", "precomputed"],
            ),
            IntField(
                name='degree_svm',
                label='degree',
                description='Degree of the polynomial kernel function (‘poly’). Ignored by all other kernels.',
                default=3,
            ),
            ChoiceField(
                name='gamma_svm',
                label='gamma',
                description='Kernel coefficient for rbf, poly and sigmoid kernels.',
                default="scale",
                choices=["scale", "auto"],
            ),
            FloatField(
                name='coef0_svm',
                label='coef0',
                description='Independent term in kernel function. It is only significant in poly and sigmoid.',
                default=0.0,
            ),
            FloatField(
                name='tol_svm',
                label='tol',
                description='Tolerance for stopping criterion.',
                default=1e-3,
            ),
            FloatField(
                name='nu_svm',
                label='nu',
                description='An upper bound on the fraction of training errors and a lower bound of the fraction of support vectors. \
                             Should be in the interval (0, 1].',
                default=0.5,
            ),
            BoolField(
                name='shrinking_svm',
                label='shrinking',
                description='Whether to use the shrinking heuristic.',
                default=True,
            ),
            IntField(
                name='max_iter_svm',
                label='max_iter',
                description='Hard limit on iterations within solver, or -1 for no limit.',
                default=-1,
            ),
        ],
    },
    section='SETTINGS'
) %}
{% set calibrated = BoolField(
    name='calibrated',
    label='Calibrate algorithm predictions',
    description='Calibrate the prediction probabilities, eliminating model-imparted bias.',
    default=True,
    section='SETTINGS',
) %}
{% set cv_algorithm = ChoiceField(
    name='cv_algorithm',
    label='Cross-Validation Algorithm',
    description='Select a cross-validation method for training and evaluating the pipeline, and for making predictions. \
                 StratifiedGroupKFold or RepeatedStratifiedGroupKFold are recommended because they will maintain class ratios \
                 across train/validation splits (stratification of labels) and will group compounds by the first 14 characters of their \
                 InChI Keys to avoid compounds with multiple entries from appearing in both the train and validation sets. \
                 If repeated cross-validation is selected, 5 independent rounds of random splits will be conducted, yielding \
                 multiple predictions per compound, which can be evaluated for consistency.',
    default='RepeatedStratifiedGroupKFold',
    choices={
        'KFold': 'sk.model_selection.KFold',
        'GroupKFold': 'sk.model_selection.GroupKFold',
        'RepeatedKFold': 'sk.model_selection.RepeatedKFold',
        'StratifiedKFold': 'sk.model_selection.StratifiedKFold',
        'StratifiedGroupKFold': 'StratifiedGroupKFold',
        'RepeatedStratifiedKFold': 'sk.model_selection.RepeatedStratifiedKFold',
        'RepeatedStratifiedGroupKFold': 'RepeatedStratifiedGroupKFold'
    },
    section='SETTINGS',
) %}
{% set cross_validation_n_folds = IntField(
    name='cross_validation_n_folds',
    label='Number of Cross-Validated Folds',
    description='Cross-validation is employed as a strategy to train the model on data that the model has not seen before, more folds will ensure that the model is generalizing well.',
    default=5,
    min=2,
    max=10,
    section='SETTINGS'
) %}
{% set hyper_param_search = ChoiceField(
    name='hyper_param_search',
    label='Hyper Parameter Search Type',
    default='None',
    description='Hyper parameter searching is used to automatically select the best parameters (using the primary metric as the criteria).',
    choices={
        'None': 'None',
        'RandomizedSearchCV': 'sk.model_selection.RandomizedSearchCV',
        'GridSearchCV': 'sk.model_selection.GridSearchCV',
    },
    section='SETTINGS'
) %}
{% set primary_metric = ChoiceField(
    name='primary_metric',
    label='Primary Evaluation Metric',
    default='roc_auc',
    description='The primary evaluation metric is used for deciding how we assess the performance of our model. \
                 Area under the receiver operating characteristic curve (roc_auc) is recommended for most tasks.',
    choices=[
        'accuracy',
        'adjusted_mutual_info_score',
        'adjusted_rand_score',
        'average_precision',
        'balanced_accuracy',
        'completeness_score',
        'explained_variance',
        'f1',
        'f1_macro',
        'f1_micro',
        'f1_weighted',
        'fowlkes_mallows_score',
        'homogeneity_score',
        'jaccard',
        'jaccard_macro',
        'jaccard_micro',
        'jaccard_weighted',
        'max_error',
        'mutual_info_score',
        'neg_brier_score',
        'neg_log_loss',
        'neg_mean_absolute_error',
        'neg_mean_squared_error',
        'neg_mean_squared_log_error',
        'neg_median_absolute_error',
        'neg_root_mean_squared_error',
        'normalized_mutual_info_score',
        'precision',
        'precision_macro',
        'precision_micro',
        'precision_weighted',
        'r2',
        'recall',
        'recall_macro',
        'recall_micro',
        'recall_weighted',
        'roc_auc',
        'roc_auc_ovo',
        'roc_auc_ovo_weighted',
        'roc_auc_ovr',
        'roc_auc_ovr_weighted',
        'v_measure_score'
    ],
    section='SETTINGS'
) %}
{% set evaluation_metrics = MultiChoiceField(
    name='evaluation_metrics',
    label='Evaluation Metrics',
    default=[],
    description='Additional evaluation metrics can be specified, these metrics will also be reported for all models trained.',
    value=[],
    choices=[
        'accuracy',
        'adjusted_mutual_info_score',
        'adjusted_rand_score',
        'average_precision',
        'balanced_accuracy',
        'completeness_score',
        'explained_variance',
        'f1',
        'f1_macro',
        'f1_micro',
        'f1_weighted',
        'fowlkes_mallows_score',
        'homogeneity_score',
        'jaccard',
        'jaccard_macro',
        'jaccard_micro',
        'jaccard_weighted',
        'max_error',
        'mutual_info_score',
        'neg_brier_score',
        'neg_log_loss',
        'neg_mean_absolute_error',
        'neg_mean_squared_error',
        'neg_mean_squared_log_error',
        'neg_median_absolute_error',
        'neg_root_mean_squared_error',
        'normalized_mutual_info_score',
        'precision',
        'precision_macro',
        'precision_micro',
        'precision_weighted',
        'r2',
        'recall',
        'recall_macro',
        'recall_micro',
        'recall_weighted',
        'roc_auc',
        'roc_auc_ovo',
        'roc_auc_ovo_weighted',
        'roc_auc_ovr',
        'roc_auc_ovr_weighted',
        'v_measure_score'
    ],
    section='SETTINGS',
) %}
{% set all_metrics = [primary_metric.value] + evaluation_metrics.value %}

In [ ]:
%%appyter code_hide

{% set algorithm_code = {
    'GradientBoostingClassifier': 'sk.ensemble.GradientBoostingClassifier(loss="{}", learning_rate={}, n_estimators={}, subsample={}, criterion="{}", tol={})',
    'RandomForestClassifier': 'sk.ensemble.RandomForestClassifier(n_estimators={}, criterion="{}", min_samples_split={}, min_samples_leaf={}, max_features={}, min_impurity_decrease={}, n_jobs=-1, class_weight={}, ccp_alpha={})',
    'AdaBoostClassifier': 'sk.ensemble.AdaBoostClassifier(sk.tree.DecisionTreeClassifier(max_depth={}), n_estimators={}, learning_rate={}, algorithm="{}")',
    'ExtraTreesClassifier': 'sk.ensemble.ExtraTreesClassifier(n_estimators={}, criterion="{}", min_samples_split={}, min_samples_leaf={}, max_features={}, min_impurity_decrease={}, n_jobs=-1, class_weight={}, ccp_alpha={})',
    'DecisionTreeClassifier': 'sk.tree.DecisionTreeClassifier(criterion="{}", splitter="{}", min_samples_split={}, min_samples_leaf={}, max_features={}, min_impurity_decrease={}, class_weight={}, ccp_alpha={})',
    'KNeighborsClassifier': 'sk.neighbors.KNeighborsClassifier(n_neighbors={}, weights="{}", algorithm="{}", leaf_size={}, p={}, metric="{}", n_jobs=-1)',
    'RadiusNeighborsClassifier': 'sk.neighbors.RadiusNeighborsClassifier(radius={}, weights="{}", algorithm="{}", leaf_size={}, p={}, metric="{}", outlier_label="most_frequent", n_jobs=-1)',
    'MLPClassifier': 'sk.neural_network.MLPClassifier(hidden_layer_sizes={}, activation="{}", solver="{}", alpha={}, learning_rate="{}", learning_rate_init={}, power_t={}, max_iter={}, tol={}, early_stopping={}, validation_fraction={})',
    'OneClassSVM': 'sk.svm.OneClassSVM(kernel="{}", degree={}, gamma="{}", coef0={}, tol={}, nu={}, shrinking={}, max_iter={})',
} %}

In [ ]:
%%appyter markdown

We apply a {% if hyper_param_search.value != 'None' %}{{ hyper_param_search.raw_value }} search for the hyper parameters
of a {% endif %}sklearn pipeline with a dimensionality reduction step of {{ dimensionality_reduction.raw_value }}
{% if feature_selection.value != 'None' %}and a feature selection step of {{ feature_selection.raw_value }}
{% endif %} and a{% if calibrated.value %} calibrated{%endif %} {{ algorithm.raw_value }} classifier
using {{ cross_validation_n_folds.value }}-fold {{ cv_algorithm.raw_value }} cross-validation,
optimizing {{ primary_metric.value }}{% if evaluation_metrics.value %} and computing {{ ', '.join(evaluation_metrics.value) }}{% endif %}.

This will take a long time as we are evaluating n_iter different models n_splits different times each computing all the metrics on `product(X.shape)` data points--not to mention the size of each model dictated by the range of parameters specified in the params dict.

In [ ]:
%%appyter code_exec
{% if algorithm.raw_value == 'GradientBoostingClassifier' %}
## Early stopping function
def early_stopping(n_rounds, tol=0.001):
    def early_stopping_func(i, self, local):
        rounds = getattr(self, '__rounds', 0)
        last = getattr(self, '__last', None)
        current = self.train_score_[i]
        if last and current and abs(current - last) < tol:
            rounds += 1
            if rounds > n_rounds:
                return True
        else:
            rounds = 0
        setattr(self, '__last', current)
        setattr(self, '__rounds', rounds)
        return False
    return early_stopping_func
{% endif %}

{#
param_grid = {
    'reduce_dim__n_components': randint(2, 1024),
{% if algorithm.raw_value == 'GradientBoostingClassifier' %}
    'clf__loss': ['deviance', 'exponential'],
    'clf__learning_rate': randfloat(0.001, 1.),
    'clf__subsample': randfloat(0.01, 1.),
{% elif algorithm.raw_value == 'RandomForestClassifier' %}
    'clf__oob_score': [True],
    'clf__criterion': ['gini', 'entropy'],
{% endif %}
    'clf__n_estimators': randint(10, 200),
    'clf__max_depth': randint(20, 50),
    'clf__max_features': ['sqrt', 'log2', None],
    'clf__min_impurity_decrease': randfloat(0., 0.2),
    'clf__min_weight_fraction_leaf': randfloat(0., 0.5),
}

fit_params = {
{% if algorithm.raw_value == 'GradientBoostingClassifier' %}
    'clf__monitor': early_stopping(5),
{% endif %}
}
#}

{% if cv_algorithm.raw_value in ['RepeatedStratifiedKFold', 'RepeatedStratifiedGroupKFold'] %}
n_repeats=5
{% endif %}
cv = {{ cv_algorithm }}(
    n_splits={{ cross_validation_n_folds }},
    {% if cv_algorithm.raw_value in ['RepeatedStratifiedKFold', 'RepeatedStratifiedGroupKFold'] %}
    n_repeats=n_repeats,
    {% else %}
    shuffle=True,
    {% endif %}
    random_state=rng,
)

{% if cv_algorithm.raw_value in ['GroupKFold', 'StratifiedGroupKFold', 'RepeatedStratifiedGroupKFold'] %}
groups=[key[:14] for key in X.index]    # Group compounds by atom connectivity
{% endif %}

# Scoring parameters
primary_metric = '{{ primary_metric }}'
evaluation_metrics = {{ evaluation_metrics }}
scoring_params = {k: metrics.get_scorer(k)
                  for k in [primary_metric, *evaluation_metrics]}

In [ ]:
%%appyter code_exec
{% if hyper_param_search.value == 'None' %}

df_results = pd.DataFrame()

# Store performance on each split for computing ROC and PRC curves
fprs = []
tprs = []
precs = []
recs = []

# Store cross-validation test predictions and folds
y_proba_cv = [[] for _ in range(len(y))]
folds_cv = [[] for _ in range(len(y))]

# Store models
models = []

{% if cv_algorithm.raw_value in ['GroupKFold', 'StratifiedGroupKFold', 'RepeatedStratifiedGroupKFold'] %}
groups=[key[:14] for key in X.index]    # Group compounds by atom connectivity
for fold, (train, test) in tqdm(enumerate(cv.split(X.values, y, groups=groups))):
{% else %}
for fold, (train, test) in tqdm(enumerate(cv.split(X.values, y))):
{% endif %}
    model =
    {%- if hyper_param_search.value != 'None' %} {{ hyper_param_search }}({% endif -%}
            sk.pipeline.Pipeline([
                {%- if dimensionality_reduction.value != 'None' %}
                ('reduce_dim', {{ dimensionality_reduction }}),
                {% endif %}
                {%- if feature_selection.value != 'None' %}
                ('feature_selection', {{ feature_selection }}),
                {% endif %}
                ('clf', {% if algorithm.raw_value == 'MLPClassifier' %}{{ algorithm_code.get(algorithm.raw_value).format(algorithm.value[0].value|str_to_tuple, *algorithm.value[1:]) }}
                        {% elif algorithm.raw_value in ['DecisionTreeClassifier', 'RandomForestClassifier', 'ExtraTreesClassifier'] %}{{ algorithm_code.get(algorithm.raw_value).format(algorithm.value[0].value, algorithm.value[1].value, algorithm.value[2].value|int_or_float, algorithm.value[3].value|int_or_float, *algorithm.value[4:]) }}
                        {% else %}{{ algorithm_code.get(algorithm.raw_value).format(*algorithm.value) }}{% endif %}
                ),
            ])
    {%- if hyper_param_search.value != 'None' %}){% endif %}
    model.fit(X.values[train], y[train])
    
    {% if calibrated.value %}
    calibrator = sk.calibration.CalibratedClassifierCV(model, cv='prefit')
    calibrator.fit(X.values[test], y[test])
    model = calibrator
    {% endif %}
    
    {% for metric in all_metrics %}
    df_results.loc[fold, '{{ metric }}'] = scoring_params['{{ metric }}'](model, X.values[test], y[test])
    {% endfor %}
    
    y_proba = model.predict_proba(X.values[test]) # Probability prediction will be True
    for i in range(len(test)):
        y_proba_cv[test[i]].append(y_proba[i, 1])
        folds_cv[test[i]].append(fold % {{ cross_validation_n_folds }})
    model_fpr, model_tpr, _ = metrics.roc_curve(y[test], y_proba[:, 1])
    model_prec, model_rec, _ = metrics.precision_recall_curve(y[test], y_proba[:, 1])
    fprs.append(model_fpr)
    tprs.append(model_tpr)
    precs.append(model_prec)
    recs.append(model_rec)
    models.append(model)

assert not(any(len(probs) == 0 for probs in y_proba_cv)), 'All probabilities should have been calculated'

display(df_results.agg(['mean', 'std']))
{% else %}
model.fit(X.values, y)
df_results = model.cv_results_
{% endif %}

This visualization shows the cross-validated performance of the model. Low fold variance and high AUC is desired in a well-generalized model.
* ROC curve: [roc.svg](./roc.svg)
* Precision-recall curve: [prc.svg](./prc.svg)
* Confusion matrix: [confusion_matrix.svg](./confusion_matrix.svg)

In [ ]:
%%appyter code_exec

fig, ax = plt.subplots()

tprs_interp = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)

for fold, (fpr, tpr) in enumerate(zip(fprs, tprs)):
    tpr_interp = np.interp(mean_fpr, fpr, tpr)
    tpr_interp[0] = 0.
    roc_auc = metrics.auc(fpr, tpr)
    tprs_interp.append(tpr_interp)
    aucs.append(roc_auc)
    {% if cv_algorithm.raw_value in ['RepeatedStratifiedKFold', 'RepeatedStratifiedGroupKFold'] %}
    ax.plot(fpr, tpr, alpha=0.4)
    {% else %}
    ax.plot(fpr, tpr, alpha=0.4, label='ROC Fold %d (AUC=%0.3f)' % (fold, roc_auc))
    {% endif %}

mean_tpr = np.mean(tprs_interp, axis=0)
mean_tpr[-1] = 1.0
mean_auc = sk.metrics.auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
ax.plot(mean_fpr, mean_tpr, color='b',
         label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
         lw=2, alpha=.8)

std_tpr = np.std(tprs_interp, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2)

ax.plot([0,1],[0,1],'--', label='Random')
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.legend()
plt.savefig('roc.svg')
plt.show()

z = (mean_auc - 0.5)/std_auc
cl = sp.stats.norm.cdf(z) * 100
ci = sp.stats.norm.interval(0.95, loc=mean_auc, scale=std_auc)
print('Confidence interval (95%)', ci)
print("We are %0.3f %% confident the model's results are not just chance." % (cl))
if cl > 95:
    print('This is statistically significant. These results can be trusted.')
else:
    print('This is not statistically significant. These results should not be trusted.')

In [ ]:
%%appyter code_exec

fig, ax = plt.subplots()

precs_interp = []
prc_aucs = []
mean_rec = np.linspace(0, 1, 100)

for fold, (rec, prec) in enumerate(zip(recs, precs)):
    prec_interp = np.interp(mean_rec, rec[::-1], prec[::-1])
    prc_auc = metrics.auc(rec, prec)
    precs_interp.append(prec_interp)
    prc_aucs.append(prc_auc)
    {% if cv_algorithm.raw_value in ['RepeatedStratifiedKFold', 'RepeatedStratifiedGroupKFold'] %}
    ax.plot(rec, prec, alpha=0.4)
    {% else %}
    ax.plot(rec, prec, alpha=0.4, label='PRC Fold %d (AUC=%0.3f)' % (fold, prc_auc))
    {% endif %}
    
mean_prec = np.mean(precs_interp, axis=0)
mean_auc = sk.metrics.auc(mean_rec, mean_prec)
std_auc = np.std(prc_aucs)
ax.plot(mean_rec, mean_prec, color='b',
         label=r'Mean PRC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
         lw=2, alpha=.8)

std_prec = np.std(precs_interp, axis=0)
precs_upper = np.minimum(mean_prec + std_prec, 1)
precs_lower = np.maximum(mean_prec - std_prec, 0)
plt.fill_between(mean_rec, precs_lower, precs_upper, color='grey', alpha=.2)

ax.set_xlabel('Recall')
ax.set_ylabel('Precision')
ax.legend()
plt.savefig('prc.svg')
plt.show()

In [ ]:
plt.title('Confusion Matrix (Cross-Validation)')
sns.heatmap(
    metrics.confusion_matrix(y, np.array([np.mean(probs) for probs in y_proba_cv]) > 0.5),
    annot=True,
    cmap=plt.cm.Blues,
    fmt='g'
)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.savefig('confusion_matrix.svg')
plt.show()

## Examine drug predictions

Using the binary classification model, we can rank the drug hits by their predicted score. The model can also be used to identify additional drugs that are likely to share properties with the hits. The results table is available at [drug_cv_predictions.csv](./drug_cv_predictions.csv).

### Plot distribution of predictions for positive and negative classes

In [ ]:
%%appyter code_exec

# Calculate mean and deviation of predictions
y_probas = np.array([np.mean(probs) for probs in y_proba_cv])
{% if cv_algorithm.raw_value in ['RepeatedStratifiedKFold', 'RepeatedStratifiedGroupKFold'] %}
y_probas_std = np.array([np.std(probs) for probs in y_proba_cv])
# Find minimum non-zero standard deviation to avoid dividing by zero when computing t-statistic
min_y_probas_std = max(np.min(y_probas_std[y_probas_std != 0]), 1e-10)
t_stats = (y_probas - np.mean(y_probas)) / (np.maximum(y_probas_std, min_y_probas_std)/np.sqrt(n_repeats))
# Calculate p-value using one-sample t-test
p_vals_t = 1-sp.stats.t(n_repeats-1).cdf(t_stats)
{% endif %}

In [ ]:
%%appyter code_exec

{% if cv_algorithm.raw_value in ['RepeatedStratifiedKFold', 'RepeatedStratifiedGroupKFold'] %}
y_probas_means_5 = []
y_probas_values = np.array(y_proba_cv).flatten()

np.random.seed(rng)
for i in tqdm(range(100000)):
    y_probas_means_5.append(np.mean(np.random.choice(y_probas_values, n_repeats)))
    
y_probas_means_5 = np.array(sorted(y_probas_means_5))
{% endif %}

In [ ]:
%%appyter code_exec

{% if cv_algorithm.raw_value in ['RepeatedStratifiedKFold', 'RepeatedStratifiedGroupKFold'] %}
y_probas_ts_5 = []
mean_y_probas = np.mean(y_probas)
y_probas_values = np.array(y_proba_cv).flatten()

np.random.seed(rng)
for i in tqdm(range(100000)):
    sample = np.random.choice(y_probas_values, n_repeats)
    y_probas_ts_5.append((np.mean(sample) - mean_y_probas) / (np.maximum(np.std(sample), min_y_probas_std)/np.sqrt(n_repeats)))
    
y_probas_ts_5 = np.array(sorted(y_probas_ts_5))
{% endif %}

In [ ]:
%%appyter code_exec

{% if cv_algorithm.raw_value in ['RepeatedStratifiedKFold', 'RepeatedStratifiedGroupKFold'] %}
max_mean = np.max(y_probas_means_5)
p_vals = np.array(list(tqdm((1 - np.argwhere(y_probas_means_5 >= min(pred, max_mean))[0][0] / len(y_probas_means_5)
                             for pred in y_probas), total=len(y_probas))))
{% endif %}

In [ ]:
%%appyter code_exec

{% if cv_algorithm.raw_value in ['RepeatedStratifiedKFold', 'RepeatedStratifiedGroupKFold'] %}
max_t = np.max(y_probas_ts_5)
p_vals_t_sim = np.array(list(tqdm((1 - np.argwhere(y_probas_ts_5 >= min(t, max_t))[0][0] / len(y_probas_ts_5)
                                   for t in t_stats), total=len(t_stats))))
{% endif %}

In [ ]:
%%appyter code_exec

sns.kdeplot(y_probas[y == 0], shade=True, gridsize=2000, clip=[np.min(y_probas), np.percentile(y_probas, 99.9)], label='Not known NSAID')
sns.kdeplot(y_probas[y == 1], shade=True, gridsize=2000, clip=[np.min(y_probas), np.percentile(y_probas, 99.9)], bw=0.005, label='Known NSAID')
{% if cv_algorithm.raw_value in ['RepeatedStratifiedKFold', 'RepeatedStratifiedGroupKFold'] %}
sns.kdeplot(y_probas_means_5, shade=True, gridsize=2000, clip=[np.min(y_probas), np.percentile(y_probas, 99.9)], bw=0.01, label='Null distribution\n(simulated)')
{% endif %}
plt.xlabel('Mean Predicted Probability')
plt.xlim([np.min(y_probas), np.percentile(y_probas, 99)])
plt.legend()
plt.savefig('mean-prediction-distribution-kde.svg')
plt.show()

In [ ]:
%%appyter code_exec

{% if cv_algorithm.raw_value in ['RepeatedStratifiedKFold', 'RepeatedStratifiedGroupKFold'] %}
sns.kdeplot(t_stats[y == 0], shade=True, gridsize=1000, clip=(-20, 20), bw=0.1, label='Not known NSAID')
sns.kdeplot(t_stats[y == 1], shade=True, gridsize=1000, clip=(-20, 20), bw=0.05, label='Known NSAID')
sns.kdeplot(y_probas_ts_5, shade=True, gridsize=1000, clip=(-20, 20), bw=0.1, label='Null distribution\n(simulated)')
plt.xlabel('t-statistic')
plt.xlim([-20,20])
plt.legend()
plt.savefig('t-statistic-distribution-kde.svg')
plt.show()
{% endif %}

### Overlay predictions on visualization of input space

In [ ]:
%%appyter code_exec

# Add attributes for plotting to Dataframe
X_reduced_df['Predicted Probability'] = y_probas
X_reduced_df['log10(pred)'] = np.log10(y_probas + 1e-10)
{% if cv_algorithm.raw_value in ['RepeatedStratifiedKFold', 'RepeatedStratifiedGroupKFold'] %}
X_reduced_df['p-value'] = p_vals_t_sim
X_reduced_df['log10(p-value)'] = np.log10(X_reduced_df['p-value'])
X_reduced_df['Standard Deviation'] = y_probas_std
{% endif %}
X_reduced_df['Cross-validation fold'] = folds_cv
{% if cv_algorithm.raw_value in ['RepeatedStratifiedKFold', 'RepeatedStratifiedGroupKFold'] %}
X_reduced_df['marker size'] = 2*np.minimum(2-np.log10(X_reduced_df['p-value']), 5)
{% else %}
max_p, min_p = np.min(-X_reduced_df['log10(pred)']), np.max(-X_reduced_df['log10(pred)'])
X_reduced_df['marker size'] = (-X_reduced_df['log10(pred)'] - min_p) / (max_p - min_p) * 6 + 4
{% endif %}
X_reduced_df['text'] = ['<br>'.join(['Drug Name: ' + str(name),
                                     'InChI Key: ' + str(inchi),
                                     'Predicted Probability: {:.1e}'.format(p),
                                     {% if cv_algorithm.raw_value in ['RepeatedStratifiedKFold', 'RepeatedStratifiedGroupKFold'] %}
                                     'Standard Deviation: {:.1e}'.format(s),
                                     'p-value: {:.1e}'.format(p_val),
                                     {% endif %}
                                     'Label: ' + str(label),
                                     'Cross-validation fold: ' + str(fold)])
                  for name, inchi, p, {% if cv_algorithm.raw_value in ['RepeatedStratifiedKFold', 'RepeatedStratifiedGroupKFold'] %}s, p_val, {% endif %}label, fold in zip(X_reduced_df['Drug Name'],
                                                         X_reduced_df['InChI Key'],
                                                         X_reduced_df['Predicted Probability'],
                                                         {% if cv_algorithm.raw_value in ['RepeatedStratifiedKFold', 'RepeatedStratifiedGroupKFold'] %}
                                                         X_reduced_df['Standard Deviation'],
                                                         X_reduced_df['p-value'],
                                                         {% endif %}
                                                         X_reduced_df['Label'],
                                                         X_reduced_df['Cross-validation fold'])]
X_reduced_df.to_csv('X_reduced_df.csv')

# Helper function for formatting Plotly colorbar
def colorbar_param(values_log10, **kwargs):
    min_val = np.floor(np.min(values_log10))
    max_val = np.ceil(np.max(values_log10))
    
    ticks1 = 10**np.arange(min_val, max_val+1)
    ticks2 = 3*10**np.arange(min_val, max_val)
    
    ticktext = sorted(np.concatenate([ticks1, ticks2]))
    tickvals = list(np.log10(ticktext))
    ticktext = ['{:.0e}'.format(text) for text in ticktext]
    
    return dict(ticktext=ticktext, tickvals=tickvals, **kwargs)

fig = go.Figure()
for label in sorted(set(X_reduced_df['Label'])):
    X_plot = X_reduced_df[X_reduced_df['Label'] == label].sort_values(['Predicted Probability'])
    fig.add_trace(go.Scatter(mode='markers',
                               x=X_plot['Component 1'], y=X_plot['Component 2'],
                               text=X_plot['text'],
                               name=label,
                               marker=dict(
                                   color=X_plot['log10(pred)'],
                                   cmin=np.percentile(X_reduced_df['log10(pred)'], 50),
                                   cmax=np.max(X_reduced_df['log10(pred)']),
                                   size=X_plot['marker size'],
                                   colorbar=colorbar_param(X_plot['log10(pred)'], title='Predicted Probability'),
                                   symbol=X_plot['marker symbol'],
                                   line_width=1,
                                   colorscale='plasma'
                               )))
fig.update_layout(height=600, width=800,
                  xaxis_title='Component 1',
                  yaxis_title='Component 2',
                  title_text='Predicted Probabilities ({{ visualization_reduction.raw_value }})',
                  legend_title_text='Target Label',
                  legend=dict(
                      yanchor="top",
                      y=0.98,
                      xanchor="left",
                      x=0.02
                  ),
                  template='simple_white')
fig.show()

### Tables of top-predicted compounds

In [ ]:
%%appyter code_exec
# Obtain prediction results
results = pd.DataFrame(np.array([
    querysepl1000fwd.get_drug_names(X.index),
    Drugmonizome.get_drug_names(X.index),
    folds_cv,
    y,
    y_probas,
    {% if cv_algorithm.raw_value in ['RepeatedStratifiedKFold', 'RepeatedStratifiedGroupKFold'] %}
    y_probas_std,
    t_stats,
    p_vals,
    p_vals_t,
    p_vals_t_sim,
    {% endif %}
]).T, columns=[
    'Name (L1000FWD)',
    'Name (Drugmonizome)',
    'Cross-validation fold',
    'Known',
    'Prediction Probability',
    {% if cv_algorithm.raw_value in ['RepeatedStratifiedKFold', 'RepeatedStratifiedGroupKFold'] %}
    'Prediction Probability Std. Dev.',
    't statistic',
    'p value (simulated mean distribution)',
    'p value (one sample t test)',
    'p value (simulated t distribution)',
    {% endif %}
], index=X.index).astype({'Known': 'bool',
                          'Prediction Probability': 'float64',
                          {% if cv_algorithm.raw_value in ['RepeatedStratifiedKFold', 'RepeatedStratifiedGroupKFold'] %}
                          'Prediction Probability Std. Dev.': 'float64',
                          't statistic': 'float64',
                          'p value (simulated mean distribution)': 'float64',
                          'p value (one sample t test)': 'float64',
                          'p value (simulated t distribution)': 'float64',{% endif %}})

results.to_csv('drug_cv_predictions.csv')

In [ ]:
# Rank known drug hits
df = results[((results['Known'] == 1))]
show(df.reset_index(), maxBytes=0, order=[[ 5, "desc" ]], columnDefs=[{'width': '110px', 'targets': [0, 1]}])

In [ ]:
# Predict additional drugs
df = results[results['Known'] == 0]
show(df.reset_index(), maxBytes=0, order=[[ 5, "desc" ]], columnDefs=[{'width': '110px', 'targets': [0, 1]}])

## Examine feature importances

The relative contribution of each input feature to the final model predictions can be estimated for recursive feature selection and for a variety of tree-based models. Note that this analysis is not available if a dimensionality reduction algorithm is used.

In [ ]:
%%appyter markdown

{% if feature_selection.raw_value == 'RecursiveSelectionFromExtraTrees' and dimensionality_reduction.raw_value == 'None' %}
When recursive feature selection is performed, the features are ranked by the stage at which they were removed.
Selected (i.e. estimated best) features are have importance 1. The ranks are averaged across cross-validation
splits to produce an average importance score. The full feature importance table is available at
[feature_importance.csv](./feature_importance.csv).
{% endif %}

In [ ]:
%%appyter code_exec

{% if feature_selection.raw_value == 'RecursiveSelectionFromExtraTrees' and dimensionality_reduction.raw_value == 'None' %}
all_rankings = []
{% endif %}
{% if algorithm.raw_value in ['GradientBoostingClassifier', 'RandomForestClassifier', 'AdaBoostClassifier', 'ExtraTreesClassifier', 'DecisionTreeClassifier'] %}
all_feature_importances = []
{% endif %}
for model in models:
    {% if calibrated.value %}
    for calibrated_clf in model.calibrated_classifiers_:
        pipeline = calibrated_clf.base_estimator
    {% else %}
        pipeline = model
    {% endif %}
        
        {% if feature_selection.raw_value == 'RecursiveSelectionFromExtraTrees' %}
        ranking = pipeline['feature_selection'].ranking_
        all_rankings.append(ranking)
        {% endif %}
        
        {% if algorithm.raw_value in ['GradientBoostingClassifier', 'RandomForestClassifier', 'AdaBoostClassifier', 'ExtraTreesClassifier', 'DecisionTreeClassifier'] %}
        {% if feature_selection.raw_value != 'None' %}
        feature_importances = np.zeros(pipeline['feature_selection'].get_support().shape)
        feature_importances[pipeline['feature_selection'].get_support()] = pipeline['clf'].feature_importances_
        {% else %}
        feature_importances = pipeline['clf'].feature_importances_
        {% endif %}
        all_feature_importances.append(feature_importances)
        {% endif %}

In [ ]:
%%appyter code_exec

{% if dimensionality_reduction.raw_value == 'None' %}
df_feat_imp = pd.DataFrame({'Feature': X.columns,
                            'Dataset': reduce(lambda a,b: a+b, ([dataset]*size for dataset, size in dataset_sizes)),
                            {% if feature_selection.raw_value == 'RecursiveSelectionFromExtraTrees' %}
                            'Ranking Mean': np.mean(all_rankings, axis=0),
                            'Ranking Std. Dev.': np.std(all_rankings, axis=0),
                            {% endif %}
                            'Importance Mean': np.mean(all_feature_importances, axis=0),
                            'Importance Std. Dev.': np.std(all_feature_importances, axis=0)})
df_feat_imp = df_feat_imp.set_index('Feature').sort_values('Importance Mean', ascending=False)
show(df_feat_imp.reset_index(), maxBytes=0, order=[[ 2, "desc"]])
df_feat_imp.to_csv('feature_importance.csv')
{% endif %}

In [ ]:
%%appyter markdown

{% if feature_selection.raw_value == 'RecursiveSelectionFromExtraTrees' and dimensionality_reduction.raw_value == 'None' %}
Plot the distribution of importance scores for features in each dataset ([feature_importance.svg](./feature_importance.svg)).
Features with lower scores were retained for more rounds during recursive feature selection
and have greater relative importance.
{% endif %}

In [ ]:
%%appyter code_exec

{% if feature_selection.raw_value == 'RecursiveSelectionFromExtraTrees' and dimensionality_reduction.raw_value == 'None' %}
fig, axs = plt.subplots(1, 2, figsize=(15, 5))
df_feat_imp = df_feat_imp.sort_values('Ranking Mean')
for dataset in set(df_feat_imp.Dataset):
    importance_scores = df_feat_imp.loc[df_feat_imp.Dataset == dataset]['Ranking Mean'].values
    importance_scores_std = df_feat_imp.loc[df_feat_imp.Dataset == dataset]['Ranking Std. Dev.'].values
    lower = importance_scores - importance_scores_std
    upper = importance_scores + importance_scores_std
    axs[0].plot(importance_scores, label=dataset)
    axs[0].fill_between(np.arange(len(importance_scores)), lower, upper, alpha=.2)
    axs[1].plot(np.linspace(0, 1, len(importance_scores)), importance_scores, label=dataset)
    axs[1].fill_between(np.linspace(0, 1, len(importance_scores)), lower, upper, alpha=.2)
for i in [0, 1]:
    axs[i].legend()
    axs[i].set_title('Distribution of feature ranking from recursive feature elimination')
    axs[i].set_ylabel('Average feature ranking\n(lower ranking is more important)')
axs[0].set_xlabel('Ranked features (absolute count)')
axs[1].set_xlabel('Ranked features (relative count)')
axs[0].set_xlim([0,512])
plt.tight_layout()
plt.savefig('feature_importance_rfe.svg')
plt.show()
{% endif %}

In [ ]:
%%appyter code_exec

{% if algorithm.raw_value in ['GradientBoostingClassifier', 'RandomForestClassifier', 'AdaBoostClassifier', 'ExtraTreesClassifier', 'DecisionTreeClassifier']  and dimensionality_reduction.raw_value == 'None' %}
fig, axs = plt.subplots(2, 2, figsize=(15, 10))
df_feat_imp = df_feat_imp.sort_values('Importance Mean', ascending=False)
for dataset in set(df_feat_imp.Dataset):
    importance_scores = df_feat_imp.loc[df_feat_imp.Dataset == dataset]['Importance Mean'].values
    importance_scores_std = df_feat_imp.loc[df_feat_imp.Dataset == dataset]['Importance Std. Dev.'].values
    lower = importance_scores - importance_scores_std
    upper = importance_scores + importance_scores_std
    axs[0][0].plot(importance_scores, label=dataset)
    axs[0][0].fill_between(np.arange(len(importance_scores)), lower, upper, alpha=.2)
    axs[0][1].plot(np.linspace(0, 1, len(importance_scores)), importance_scores, label=dataset)
    axs[0][1].fill_between(np.linspace(0, 1, len(importance_scores)), lower, upper, alpha=.2)
    
    importance_scores = np.cumsum(df_feat_imp.loc[df_feat_imp.Dataset == dataset]['Importance Mean'].values)
    axs[1][0].plot(importance_scores, label=dataset)
    axs[1][1].plot(np.linspace(0, 1, len(importance_scores)), importance_scores, label=dataset)
for i in [0, 1]:
    axs[0][i].legend()
    axs[0][i].set_title('Distribution of feature scores from model')
    axs[1][i].set_title('Cumulative distribution of feature scores from model')
    axs[i][0].set_xlabel('Ranked features (absolute count)')
    axs[i][1].set_xlabel('Ranked features (relative count)')
    axs[0][i].set_ylabel('Average feature importance\n(higher score is more important)')
    axs[1][i].set_ylabel('Cumulative sum of feature importance')
    axs[i][0].set_xlim([0,512])
plt.tight_layout()
plt.savefig('feature_importance.svg')
plt.show()
{% endif %}